#### ETL for Engage3 technical exercise 2 
#### Sanjay Mamidi 9/23/2019


#### Analysis:  
WholeFoods pricing data seems to be off for Kansas. There seems to be an auditor in  Kansas Auditor Id 713 whose 
data in consistently low. This might be a problem that needs futher investigation. 
Prices in Northern California seem higher than elsewhere in the country.
In this dataset Northern California had the lowest number of products on sale but with the highest mean prices. 
The mean prices were highest in CA followed by NY , Texas and Kansas. The highest price item also had a similar trend. 
I did not use Auditors data Load for the final analysis except for highlighting a particular Auditor

In [ ]:
import pandas as pd

In [244]:
json_df = pd.read_json("stores.json", orient="records")
auditors_df = pd.read_csv("auditors.csv",encoding='latin1')
prices_df = pd.read_csv("prices.csv",encoding='latin1')

In [245]:
json_df.head(3)

,Banner,Region,Store ID
0,Walmart,Northern California,66999
1,Trader Joes,Northern California,4698
2,Safeway,Northern California,39482


In [246]:
auditors_df.head(3)

,Auditor ID,First,Last,Region
0,234,Sue,Smith,Northern California
1,536,Bob,Smith,Northern California
2,98,Jack,Smith,New York


In [247]:
prices_df.head(5)

,Auditor ID,Date,Price,Store ID,UPC
0,234,10/18/17,24.95,66999,268588472
1,234,10/27/17,49.71,66999,475245085
2,234,10/20/17,25.75,66999,126967843
3,234,10/23/17,18.81,66999,708930835
4,234,10/23/17,33.32,66999,325885139


In [248]:
prices_df.count()

Auditor ID    12315
Date          12315
Price         12315
Store ID      12315
UPC           12315
dtype: int64

In [249]:
prices_df.nunique()

Auditor ID       8
Date            14
Price         2596
Store ID        20
UPC           1000
dtype: int64

In [250]:
prices_df = prices_df.merge(json_df, on='Store ID', how = 'inner')
prices_df.columns

Index(['Auditor ID', 'Date', 'Price', 'Store ID', 'UPC', 'Banner', 'Region'], dtype='object')

In [252]:
prices_df.head(3)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region
0,234,10/18/17,24.95,66999,268588472,Walmart,Northern California
1,234,10/27/17,49.71,66999,475245085,Walmart,Northern California
2,234,10/20/17,25.75,66999,126967843,Walmart,Northern California


In [253]:
# prices_df = prices_df.merge(auditors_df, on='Auditor ID', how = 'inner')
prices_df.columns

Index(['Auditor ID', 'Date', 'Price', 'Store ID', 'UPC', 'Banner', 'Region'], dtype='object')

In [254]:
prices_P_df = prices_df.pivot(columns ='Region', values = 'Price')

In [255]:
prices_df.nunique()

Auditor ID       8
Date            14
Price         2593
Store ID        18
UPC           1000
Banner           5
Region           4
dtype: int64

In [256]:
RegionArray = prices_df.Region.unique()
RegionArray

array(['Northern California', 'New York', 'Kansas', 'Texas'], dtype=object)

In [257]:
prices_df_NC = prices_df[prices_df.Region == 'Northern California']
prices_P_df = prices_df_NC.pivot(columns ='Region', values = 'Price')
prices_df_NC = pd.concat([prices_df_NC,prices_P_df], axis = 1)
prices_df_NC.tail(3)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region,Northern California
1631,536,10/25/17,59.69,34957,436463728,Whole Foods,Northern California,59.69
1632,536,10/22/17,30.89,34957,376153383,Whole Foods,Northern California,30.89
1633,536,10/23/17,49.19,34957,514912132,Whole Foods,Northern California,49.19


In [258]:
prices_df_NY = prices_df[prices_df.Region == 'New York']
prices_P_df = prices_df_NY.pivot(columns ='Region', values = 'Price')
prices_df_NY = pd.concat([prices_df_NY,prices_P_df], axis = 1)
prices_df_NY.tail(3)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region,New York
4656,98,10/24/17,21.69,50948,352259734,Whole Foods,New York,21.69
4657,98,10/18/17,26.39,50948,627807402,Whole Foods,New York,26.39
4658,98,10/21/17,6.59,50948,866698043,Whole Foods,New York,6.59


In [259]:
prices_df_K = prices_df[prices_df.Region == 'Kansas']
prices_P_df = prices_df_K.pivot(columns ='Region', values = 'Price')
prices_df_K = pd.concat([prices_df_K,prices_P_df], axis = 1)
prices_df_K.tail(3)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region,Kansas
7743,713,10/25/17,1.99,39287,125829867,Whole Foods,Kansas,1.99
7744,713,10/21/17,1.99,39287,912873332,Whole Foods,Kansas,1.99
7745,713,10/21/17,1.99,39287,163809044,Whole Foods,Kansas,1.99


In [260]:
prices_df_T = prices_df[prices_df.Region == 'Texas']
prices_P_df = prices_df_T.pivot(columns ='Region', values = 'Price')
prices_df_T = pd.concat([prices_df_T,prices_P_df], axis = 1)
prices_df_T.nunique()


Auditor ID       2
Date            14
Price         1156
Store ID         5
UPC           1000
Banner           5
Region           1
Texas         1156
dtype: int64

In [261]:
prices_df_T.head(3)

,Auditor ID,Date,Price,Store ID,UPC,Banner,Region,Texas
7746,63,10/23/17,37.85,50495,311513952,Walmart,Texas,37.85
7747,63,10/28/17,16.33,50495,793865483,Walmart,Texas,16.33
7748,63,10/29/17,33.23,50495,774342015,Walmart,Texas,33.23


In [262]:
prices_final_df = pd.merge(prices_df_T, prices_df_K, on =['UPC','Banner'], how = 'left')
prices_final_df = prices_final_df.sort_values(by=['UPC'])
prices_final_df = pd.merge(prices_final_df,prices_df_NY, on =['UPC','Banner'], how = 'left') 
prices_final_df = pd.merge(prices_final_df,prices_df_NC, on =['UPC','Banner'], how = 'left') 
prices_final_df =prices_final_df.drop(columns = ['Auditor ID_x','Date_x','Store ID_x','Price_x','Region_x'])
prices_final_df =prices_final_df.drop(columns = ['Auditor ID_y','Date_y','Store ID_y','Price_y','Region_y'])
prices_final_df = prices_final_df.sort_values(by=['UPC'])
prices_final_df


,UPC,Banner,Texas,Kansas,New York,Northern California
0,11873171,Whole Foods,5.49,1.99,5.69,NaN
1,11873171,Walmart,4.75,NaN,NaN,5.53
2,11873171,Wegmans,5.09,NaN,5.19,NaN
3,11873171,Trader Joes,4.99,NaN,NaN,NaN
4,11873171,Safeway,5.19,NaN,NaN,6.09
5,15052612,Wegmans,52.89,52.39,54.49,NaN
6,15052612,Safeway,54.49,53.99,NaN,NaN
7,15052612,Whole Foods,57.79,1.99,59.49,NaN
8,16482322,Wegmans,17.59,NaN,18.09,NaN
9,16482322,Trader Joes,17.19,NaN,17.69,NaN


In [268]:
prices_final_df.describe()

,UPC,Texas,Kansas,New York,Northern California
count,3.765000e+03,3765.000000,2194.000000,2162.000000,1124.000000
mean,5.060907e+08,30.084550,23.685966,31.035546,35.263941
std,2.875843e+08,16.709557,18.503808,17.181098,19.509193
min,1.187317e+07,0.590000,0.690000,0.590000,0.690000
25%,2.592874e+08,16.290000,3.260000,16.825000,18.962500
50%,5.072036e+08,29.840000,21.440000,31.090000,34.840000
75%,7.627109e+08,44.060000,39.390000,45.342500,51.290000
max,9.991851e+08,63.890000,59.990000,65.590000,73.690000
